In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
trainA = []
trainB = []
for i in range(1,701):
  img = cv2.imread('rain/{}clean.jpg'.format(i))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  trainA.append(img)
  img = cv2.imread('rain/{}bad.jpg'.format(i))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  trainB.append(img)
trainA = np.array(trainA)
trainB = np.array(trainB)
trainA = (trainA - 127.5)/127.5
trainB = (trainB - 127.5)/127.5

In [4]:
# example of defining a u-net encoder-decoder generator model
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import UpSampling2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

# define an encoder block
def define_encoder_block(layer_in, n_filters):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	g = UpSampling2D(size=(2,2))(layer_in)
	g = Conv2DTranspose(n_filters, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(g)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
	e1 = define_encoder_block(in_image, 64)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model: C512-C512-C512-C512-C512-C256-C128-C64
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512)
	d5 = decoder_block(d4, e3, 256)
	d6 = decoder_block(d5, e2, 1280)
	d7 = decoder_block(d6, e1, 64)
	# output
	g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define image shape
image_shape = (256,256,3)
# create the model
model = define_generator(image_shape)
# summarize the model
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 3136        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 128)  131200      leaky_re_lu_1[0][0]              
_____________________________________

In [5]:
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, add, Concatenate
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras.initializers import RandomNormal
from tensorflow.keras.losses import BinaryCrossentropy
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss= BinaryCrossentropy(from_logits=True), optimizer=opt, loss_weights=[0.5])
	return model 
# define image shape
image_shape = (256,256,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()
# plot the model
#plot_model(model, to_file='/content/drive/My Drive/test/discriminator_model_plot.png', show_shapes=True, show_layer_names=True)

Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 256, 256, 6)  0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 128, 128, 64) 6208    

In [6]:
from tensorflow.keras.losses import BinaryCrossentropy
def define_gan(g_model, d_model, image_shape):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_target = Input(shape = image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model([in_src,in_target], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=[BinaryCrossentropy(from_logits=True), 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
def generate_real_samples(n_samples, patch_shape):
	# unpack dataset

	# choose random instances
	ix = np.random.randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [8]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [9]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, d_model, gan_model, n_samples=1):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# scale all pixels from [-1,1] to [0,1]
	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0
	X_fakeB = 255 * X_fakeB	
	# plot generated target image
	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_fakeB[i])
	# save plot to file
	filename1 = 'Upsampling/training/plot_%06d.png' % (step+1)
	cv2.imwrite(filename1,X_fakeB[0])
	# save the generator, discriminator and gan models
	filename2 = 'Upsampling/training/g_model_%06d.h5' % (step+1)
	g_model.save(filename2)
	#filename3 = 'sahil/training/d_model_%06d.h5' % (step+1)
	#d_model.save(filename3)
	#filename4 = 'sahil/training/gan_model_%06d.h5' % (step+1)
	#gan_model.save(filename4)
	print('>Saved: %s, and %s' % (filename1, filename2))

In [10]:
def train(d_model, g_model, gan_model, n_epochs=200, n_batch=1, n_patch=32):
	# unpack dataset
  
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples( n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realB, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realB, X_realA], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realB, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch([X_realB,X_realA], [y_fake, X_realA])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    # summarize model performance
		if (i+1) % (bat_per_epo * 1) == 0:
			summarize_performance(i, g_model,d_model, gan_model) 

In [ ]:
image_shape = (256,256,3)
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model)

Instructions for updating:
Use tf.cast instead.


/home/pytorch/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, d1[0.237] d2[0.514] g[54.188]
>2, d1[0.231] d2[0.507] g[59.581]
>3, d1[0.235] d2[0.498] g[41.289]
>4, d1[0.239] d2[0.471] g[41.360]
>5, d1[0.258] d2[0.460] g[28.388]
>6, d1[0.245] d2[0.469] g[25.183]
>7, d1[0.257] d2[0.471] g[53.610]
>8, d1[0.248] d2[0.446] g[37.801]
>9, d1[0.252] d2[0.425] g[35.672]
>10, d1[0.250] d2[0.412] g[36.454]
>11, d1[0.243] d2[0.459] g[54.847]
>12, d1[0.245] d2[0.443] g[23.553]
>13, d1[0.242] d2[0.453] g[54.246]
>14, d1[0.241] d2[0.449] g[47.433]
>15, d1[0.221] d2[0.484] g[24.880]
>16, d1[0.283] d2[0.439] g[22.172]
>17, d1[0.267] d2[0.481] g[28.480]
>18, d1[0.283] d2[0.433] g[21.537]
>19, d1[0.255] d2[0.437] g[30.590]
>20, d1[0.250] d2[0.463] g[31.119]
>21, d1[0.272] d2[0.439] g[33.481]
>22, d1[0.263] d2[0.456] g[40.405]
>23, d1[0.239] d2[0.484] g[30.593]
>24, d1[0.274] d2[0.481] g[23.272]
>25, d1[0.278] d2[0.459] g[31.784]
>26, d1[0.283] d2[0.461] g[31.761]
>27, d1[0.253] d2[0.484] g[19.334]
>28, d1[0.270] d2[0.466] g[25.024]
>29, d1[0.268] d2[0.443] g[29

>233, d1[0.307] d2[0.362] g[19.304]
>234, d1[0.305] d2[0.370] g[20.427]
>235, d1[0.321] d2[0.356] g[19.199]
>236, d1[0.320] d2[0.356] g[16.428]
>237, d1[0.329] d2[0.377] g[25.033]
>238, d1[0.314] d2[0.371] g[16.297]
>239, d1[0.311] d2[0.361] g[20.342]
>240, d1[0.323] d2[0.364] g[21.301]
>241, d1[0.296] d2[0.374] g[18.783]
>242, d1[0.312] d2[0.358] g[12.372]
>243, d1[0.311] d2[0.359] g[17.408]
>244, d1[0.319] d2[0.379] g[12.845]
>245, d1[0.307] d2[0.368] g[15.368]
>246, d1[0.311] d2[0.381] g[14.893]
>247, d1[0.303] d2[0.365] g[16.889]
>248, d1[0.318] d2[0.369] g[12.348]
>249, d1[0.333] d2[0.362] g[21.347]
>250, d1[0.322] d2[0.360] g[14.258]
>251, d1[0.326] d2[0.361] g[12.283]
>252, d1[0.325] d2[0.357] g[11.783]
>253, d1[0.327] d2[0.353] g[19.887]
>254, d1[0.319] d2[0.364] g[11.163]
>255, d1[0.312] d2[0.362] g[13.176]
>256, d1[0.299] d2[0.376] g[18.126]
>257, d1[0.332] d2[0.367] g[16.237]
>258, d1[0.317] d2[0.372] g[30.882]
>259, d1[0.326] d2[0.353] g[11.889]
>260, d1[0.312] d2[0.359] g[

>461, d1[0.291] d2[0.360] g[14.107]
>462, d1[0.299] d2[0.360] g[27.434]
>463, d1[0.279] d2[0.362] g[15.166]
>464, d1[0.290] d2[0.360] g[26.897]
>465, d1[0.276] d2[0.379] g[17.575]
>466, d1[0.287] d2[0.375] g[16.831]
>467, d1[0.288] d2[0.374] g[13.128]
>468, d1[0.272] d2[0.363] g[31.385]
>469, d1[0.299] d2[0.354] g[21.417]
>470, d1[0.275] d2[0.361] g[24.865]
>471, d1[0.312] d2[0.355] g[16.680]
>472, d1[0.295] d2[0.361] g[12.848]
>473, d1[0.286] d2[0.357] g[14.206]
>474, d1[0.296] d2[0.354] g[13.950]
>475, d1[0.271] d2[0.377] g[14.518]
>476, d1[0.299] d2[0.370] g[13.467]
>477, d1[0.265] d2[0.371] g[18.901]
>478, d1[0.312] d2[0.363] g[10.124]
>479, d1[0.275] d2[0.395] g[17.789]
>480, d1[0.282] d2[0.360] g[10.998]
>481, d1[0.291] d2[0.351] g[15.269]
>482, d1[0.294] d2[0.353] g[33.816]
>483, d1[0.277] d2[0.368] g[13.591]
>484, d1[0.282] d2[0.351] g[17.623]
>485, d1[0.275] d2[0.387] g[22.018]
>486, d1[0.261] d2[0.364] g[16.672]
>487, d1[0.282] d2[0.388] g[12.413]
>488, d1[0.265] d2[0.361] g[

...

>7658, d1[0.157] d2[0.347] g[88.963]
>7659, d1[0.157] d2[0.347] g[93.868]
>7660, d1[0.157] d2[0.347] g[108.992]
>7661, d1[0.157] d2[0.347] g[99.576]
>7662, d1[0.157] d2[0.347] g[94.353]
>7663, d1[0.157] d2[0.347] g[109.880]
>7664, d1[0.157] d2[0.347] g[106.780]
>7665, d1[0.157] d2[0.347] g[92.485]
>7666, d1[0.157] d2[0.347] g[98.792]
>7667, d1[0.157] d2[0.347] g[95.365]
>7668, d1[0.157] d2[0.347] g[90.887]
>7669, d1[0.157] d2[0.347] g[94.677]
>7670, d1[0.157] d2[0.347] g[100.868]
>7671, d1[0.157] d2[0.347] g[97.907]
>7672, d1[0.157] d2[0.347] g[91.189]
>7673, d1[0.157] d2[0.347] g[77.691]
>7674, d1[0.157] d2[0.347] g[103.321]
>7675, d1[0.157] d2[0.347] g[108.995]
>7676, d1[0.157] d2[0.347] g[92.899]
>7677, d1[0.157] d2[0.347] g[82.822]
>7678, d1[0.157] d2[0.347] g[80.942]
>7679, d1[0.157] d2[0.347] g[95.682]
>7680, d1[0.157] d2[0.347] g[107.955]
>7681, d1[0.157] d2[0.347] g[84.387]
>7682, d1[0.157] d2[0.347] g[81.290]
>7683, d1[0.157] d2[0.347] g[77.691]
>7684, d1[0.157] d2[0.347] g[90

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>7700, d1[0.157] d2[0.347] g[110.446]
>Saved: Upsampling/training/plot_007700.png, and Upsampling/training/g_model_007700.h5
>7701, d1[0.157] d2[0.347] g[108.992]
>7702, d1[0.157] d2[0.347] g[95.174]
>7703, d1[0.157] d2[0.347] g[108.992]
>7704, d1[0.157] d2[0.347] g[110.771]
>7705, d1[0.157] d2[0.347] g[89.560]
>7706, d1[0.157] d2[0.347] g[97.731]
>7707, d1[0.157] d2[0.347] g[92.706]
>7708, d1[0.157] d2[0.347] g[97.576]
>7709, d1[0.157] d2[0.347] g[77.398]
>7710, d1[0.157] d2[0.347] g[79.189]
>7711, d1[0.157] d2[0.347] g[92.706]
>7712, d1[0.157] d2[0.347] g[96.240]
>7713, d1[0.157] d2[0.347] g[84.633]
>7714, d1[0.157] d2[0.347] g[85.265]
>7715, d1[0.157] d2[0.347] g[109.887]
>7716, d1[0.157] d2[0.347] g[89.628]
>7717, d1[0.157] d2[0.347] g[85.912]
>7718, d1[0.157] d2[0.347] g[106.775]
>7719, d1[0.157] d2[0.347] g[97.731]
>7720, d1[0.157] d2[0.347] g[97.731]
>7721, d1[0.157] d2[0.347] g[92.561]
>7722, d1[0.157] d2[0.347] g[106.980]
>7723, d1[0.157] d2[0.347] g[93.487]
>7724, d1[0.157] d

>7918, d1[0.157] d2[0.347] g[94.485]
>7919, d1[0.157] d2[0.347] g[95.255]
>7920, d1[0.157] d2[0.347] g[90.179]
>7921, d1[0.157] d2[0.347] g[98.657]
>7922, d1[0.157] d2[0.347] g[100.963]
>7923, d1[0.157] d2[0.347] g[105.497]
>7924, d1[0.157] d2[0.347] g[114.842]
>7925, d1[0.157] d2[0.347] g[78.241]
>7926, d1[0.157] d2[0.347] g[93.667]
>7927, d1[0.157] d2[0.347] g[95.430]
>7928, d1[0.157] d2[0.347] g[100.701]
>7929, d1[0.157] d2[0.347] g[74.819]
>7930, d1[0.157] d2[0.347] g[97.731]
>7931, d1[0.157] d2[0.347] g[100.889]
>7932, d1[0.157] d2[0.347] g[92.201]
>7933, d1[0.157] d2[0.347] g[88.210]
>7934, d1[0.157] d2[0.347] g[91.860]
>7935, d1[0.157] d2[0.347] g[100.868]
>7936, d1[0.157] d2[0.347] g[90.357]
>7937, d1[0.157] d2[0.347] g[89.467]
>7938, d1[0.157] d2[0.347] g[97.823]
>7939, d1[0.157] d2[0.347] g[97.576]
>7940, d1[0.157] d2[0.347] g[95.798]
>7941, d1[0.157] d2[0.347] g[102.739]
>7942, d1[0.157] d2[0.347] g[94.873]
>7943, d1[0.157] d2[0.347] g[88.548]
>7944, d1[0.157] d2[0.347] g[99

>8138, d1[0.157] d2[0.347] g[115.825]
>8139, d1[0.157] d2[0.347] g[84.502]
>8140, d1[0.157] d2[0.347] g[110.771]
>8141, d1[0.157] d2[0.347] g[105.141]
>8142, d1[0.157] d2[0.347] g[100.949]
>8143, d1[0.157] d2[0.347] g[88.253]
>8144, d1[0.157] d2[0.347] g[89.467]
>8145, d1[0.157] d2[0.347] g[107.059]
>8146, d1[0.157] d2[0.347] g[89.467]
>8147, d1[0.157] d2[0.347] g[97.612]
>8148, d1[0.157] d2[0.347] g[80.398]
>8149, d1[0.157] d2[0.347] g[85.264]
>8150, d1[0.157] d2[0.347] g[114.228]
>8151, d1[0.157] d2[0.347] g[97.905]
>8152, d1[0.157] d2[0.347] g[102.571]
>8153, d1[0.157] d2[0.347] g[94.873]
>8154, d1[0.157] d2[0.347] g[88.987]
>8155, d1[0.157] d2[0.347] g[91.357]
>8156, d1[0.157] d2[0.347] g[91.189]
>8157, d1[0.157] d2[0.347] g[90.285]
>8158, d1[0.157] d2[0.347] g[97.612]
>8159, d1[0.157] d2[0.347] g[105.857]
>8160, d1[0.157] d2[0.347] g[82.190]
>8161, d1[0.157] d2[0.347] g[85.130]
>8162, d1[0.157] d2[0.347] g[82.516]
>8163, d1[0.157] d2[0.347] g[108.997]
>8164, d1[0.157] d2[0.347] g[